# 逐步控制的代理

本笔记本向您展示如何使用我们全新的低级代理API，该API支持许多功能，不仅仅是执行用户查询，还可以帮助您创建任务，迭代步骤，并控制每个步骤的输入。

### 高级代理架构

我们的“代理”由与“AgentWorkers”交互的“AgentRunner”对象组成。AgentRunners是编排者，它们存储状态（包括对话记忆），创建和维护任务，通过每个任务运行步骤，并为用户提供面向用户的高级接口。

`AgentWorker` **控制任务的逐步执行**。给定一个输入步骤，代理工作者负责生成下一个步骤。它们可以使用从任务/任务步骤对象传递下来的参数进行初始化，并对状态进行操作，但本身并不固有地存储状态。外部的`AgentRunner`负责调用`AgentWorker`并收集/聚合结果。

如果您正在构建自己的代理，您可能希望创建自己的`AgentWorker`。请参阅下面的示例！

### 笔记本演示

本笔记本向您展示如何使用代理进行逐步执行和完整执行。
- 我们将向您展示如何使用OpenAIAgent（函数调用）进行执行
-


In [ ]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai

In [ ]:
!pip install llama-index

In [ ]:
import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [ ]:
def multiply(a: int, b: int) -> int:    """将两个整数相乘，并返回结果整数"""    return a * bmultiply_tool = FunctionTool.from_defaults(fn=multiply)def add(a: int, b: int) -> int:    """将两个整数相加，并返回结果整数"""    return a + badd_tool = FunctionTool.from_defaults(fn=add)tools = [multiply_tool, add_tool]

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo")

## 测试 OpenAI 代理

有两种主要的初始化代理的方式。
- **选项 1**：初始化 `OpenAIAgent`。这是 `AgentRunner` 的一个简单子类，它在内部捆绑了 `OpenAIAgentWorker`。
- **选项 2**：使用 `OpenAIAgentWorker` 初始化 `AgentRunner`。在这种情况下，您需要导入模块并组合自己的代理。

**注意**：旧的 OpenAIAgent 仍然可以通过 `from llama_index.agent import OldOpenAIAgent` 导入。


In [ ]:
from llama_index.core.agent import AgentRunnerfrom llama_index.agent.openai import OpenAIAgentWorker, OpenAIAgent# 选项1：初始化OpenAIAgentagent = OpenAIAgent.from_tools(tools, llm=llm, verbose=True)# # 选项2：使用OpenAIAgentWorker初始化AgentRunneropenai_step_engine = OpenAIAgentWorker.from_tools(tools, llm=llm, verbose=True)agent = AgentRunner(openai_step_engine)

### 测试端到端聊天

在这里，我们重新演示通过`chat()`函数执行用户任务的端到端过程。

这将逐步迭代，直到代理完成当前任务。


In [ ]:
agent.chat("Hi")

AgentChatResponse(response='Hello! How can I assist you today?', sources=[], source_nodes=[])

In [ ]:
response = agent.chat("What is (121 * 3) + 42?")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363

=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405



In [ ]:
response

AgentChatResponse(response='The result of (121 * 3) + 42 is 405.', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[])

### 逐步执行测试

现在让我们展示较低级别的API如何运行。我们做同样的事情，但是将其分解为步骤。


In [ ]:
# 开始任务任务 = agent.create_task("121 * 3 + 42的结果是多少？")

In [ ]:
step_output = agent.run_step(task.task_id)

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363



In [ ]:
step_output

TaskStepOutput(output=AgentChatResponse(response='None', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363)], source_nodes=[]), task_step=TaskStep(task_id='920e1e67-2310-430b-bfe5-813f48c9c23c', step_id='697f90f1-f6f2-4be5-a9f9-a7239eadc094', input='What is (121 * 3) + 42?', step_state={}, next_steps={}, prev_steps={}, is_ready=True), next_steps=[TaskStep(task_id='920e1e67-2310-430b-bfe5-813f48c9c23c', step_id='e526d116-0f44-4337-915f-be55a04511b9', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)], is_last=False)

In [ ]:
step_output = agent.run_step(task.task_id)

=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405



In [ ]:
step_output = agent.run_step(task.task_id)

In [ ]:
# 显示最终响应print(step_output.is_last)

True


In [ ]:
# 步骤执行完成后，我们可以最终确定响应response = agent.finalize_response(task.task_id)print(str(response))

The result of (121 * 3) + 42 is 405.


## 测试 ReAct 代理

我们将进行相同的实验，但使用 ReAct。


In [ ]:
llm = OpenAI(model="gpt-4-1106-preview")

In [ ]:
from llama_index.core.agent import AgentRunner, ReActAgentWorker, ReActAgent

In [ ]:
# 选项1：初始化OpenAIAgentagent = ReActAgent.from_tools(tools, llm=llm, verbose=True)# # 选项2：使用ReActAgentWorker初始化AgentRunner# react_step_engine = ReActAgentWorker.from_tools(tools, llm=llm, verbose=True)# agent = AgentRunner(react_step_engine)

In [ ]:
agent.chat("Hi")

Thought: The user has greeted me, and I should respond in kind.
Response: Hello! How can I assist you today?


AgentChatResponse(response='Hello! How can I assist you today?', sources=[], source_nodes=[])

In [ ]:
response = agent.chat("What is (121 * 3) + 42?")

Thought: I need to use a tool to help me calculate the multiplication part of the question first.
Action: multiply
Action Input: {'a': 121, 'b': 3}
Observation: 363
Thought: Now that I have the result of the multiplication, I need to add 42 to it.
Action: add
Action Input: {'a': 363, 'b': 42}
Observation: 405
Thought: I can answer without using any more tools.
Response: (121 * 3) + 42 equals 405.


In [ ]:
response

AgentChatResponse(response='(121 * 3) + 42 equals 405.', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[])

In [ ]:
# 开始任务任务 = agent.create_task("121 * 3 + 42的结果是多少？")

In [ ]:
step_output = agent.run_step(task.task_id)

Thought: I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 121, 'b': 3}
Observation: 363


In [ ]:
step_output.output

AgentChatResponse(response='Observation: 363', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363)], source_nodes=[])

In [ ]:
step_output = agent.run_step(task.task_id)

Thought: Now that I have the result of the multiplication, I need to add 42 to it.
Action: add
Action Input: {'a': 363, 'b': 42}
Observation: 405


In [ ]:
step_output.output

AgentChatResponse(response='Observation: 405', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[])

In [ ]:
step_output = agent.run_step(task.task_id)

Thought: I can answer without using any more tools.
Response: (121 * 3) + 42 equals 405.


In [ ]:
step_output.output

AgentChatResponse(response='(121 * 3) + 42 equals 405.', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[])

### 列出任务

有3个任务，分别对应上面的三次运行。


In [ ]:
tasks = agent.list_tasks()
print(len(tasks))

3


In [ ]:
task_state = tasks[-1]
task_state.task.input

'What is (121 * 3) + 42?'

In [ ]:
# 获取已完成的步骤completed_steps = agent.get_completed_steps(task_state.task.task_id)

In [ ]:
len(completed_steps)

3

In [ ]:
completed_steps[0]

TaskStepOutput(output=AgentChatResponse(response='Observation: 363', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[]), task_step=TaskStep(task_id='1a71f7cb-c854-4baa-ad11-0c97460a6af2', step_id='72ed54f4-4edd-496b-bb6a-d58f4275f03e', input='What is (121 * 3) + 42?', step_state={}, next_steps={}, prev_steps={}, is_ready=True), next_steps=[TaskStep(task_id='1a71f7cb-c854-4baa-ad11-0c97460a6af2', step_id='6486cc4a-15ea-490a-b761-9dc88881fc24', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)], is_last=False)

In [ ]:
for idx in range(len(completed_steps)):
    print(f"Step {idx}")
    print(f"Response: {completed_steps[idx].output.response}")
    print(f"Sources: {completed_steps[idx].output.sources}")

Step 0
Response: Observation: 363
Sources: [ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]
Step 1
Response: Observation: 405
Sources: [ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]
Step 2
Response: (121 * 3) + 42 equals 405.
Sources: [ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]
